<a href="https://colab.research.google.com/github/neonithinar/Language_detection/blob/main/1st_attempt_Europarl_Language_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language identification using N-grams
We will be using trigrams to identify the language


In [ ]:
# Common Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import time
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils


Mount the google drive

In [ ]:
! pwd
! mkdir europarl
! cd europarl
! pwd

/content
/content


In [ ]:
#! wget https://drive.google.com/file/d/1dKnhm9bdUF7KDkvp_IpyPSguXaBBIJFa/view?usp=sharing

--2021-02-04 05:48:26--  https://docs.google.com/u/0/nonceSigner?nonce=6op4hf6pumiqe
Resolving docs.google.com (docs.google.com)... 74.125.142.138, 74.125.142.100, 74.125.142.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.138|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://docs.google.com/nonceSigner?nonce=6op4hf6pumiqe [following]
--2021-02-04 05:48:26--  https://docs.google.com/nonceSigner?nonce=6op4hf6pumiqe
Reusing existing connection to docs.google.com:443.
HTTP request sent, awaiting response... 400 Bad Request
2021-02-04 05:48:26 ERROR 400: Bad Request.



In [ ]:
lang = ['bulg', 'dani','germ', 'gree', 'span', 'esto', 'finn', 'fren', 'hung', 'ital', 'lith', 'latv', 'dutc', 'poli', 'port', 'roma', 'slvk', 'slve', 'swed']

In [ ]:
df = pd.DataFrame({'languages': ['bulg', 'dani','germ', 'gree', 'span', 'esto', 'finn', 'fren', 'hung', 'ital', 'lith', 'latv', 'dutc', 'poli', 'port', 'roma', 'slvk', 'slve', 'swed'], 
                   'paths':['/content/drive/MyDrive/Europarl/bg-en.tgz', '/content/drive/MyDrive/Europarl/da-en.tgz', '/content/drive/MyDrive/Europarl/de-en.tgz', '/content/drive/MyDrive/Europarl/el-en.tgz', '/content/drive/MyDrive/Europarl/es-en.tgz', '/content/drive/MyDrive/Europarl/et-en.tgz', '/content/drive/MyDrive/Europarl/fi-en.tgz', '/content/drive/MyDrive/Europarl/fr-en.tgz','/content/drive/MyDrive/Europarl/hu-en.tgz', '/content/drive/MyDrive/Europarl/it-en.tgz', '/content/drive/MyDrive/Europarl/lt-en.tgz', '/content/drive/MyDrive/Europarl/lv-en.tgz','/content/drive/MyDrive/Europarl/nl-en.tgz', '/content/drive/MyDrive/Europarl/pl-en.tgz', '/content/drive/MyDrive/Europarl/pt-en.tgz', '/content/drive/MyDrive/Europarl/ro-en.tgz', '/content/drive/MyDrive/Europarl/sk-en.tgz', '/content/drive/MyDrive/Europarl/sl-en.tgz', '/content/drive/MyDrive/Europarl/sv-en.tgz']})
df
                            

,languages,paths
0,bulg,/content/drive/MyDrive/Europarl/bg-en.tgz
1,dani,/content/drive/MyDrive/Europarl/da-en.tgz
2,germ,/content/drive/MyDrive/Europarl/de-en.tgz
3,gree,/content/drive/MyDrive/Europarl/el-en.tgz
4,span,/content/drive/MyDrive/Europarl/es-en.tgz
5,esto,/content/drive/MyDrive/Europarl/et-en.tgz
6,finn,/content/drive/MyDrive/Europarl/fi-en.tgz
7,fren,/content/drive/MyDrive/Europarl/fr-en.tgz
8,hung,/content/drive/MyDrive/Europarl/hu-en.tgz
9,ital,/content/drive/MyDrive/Europarl/it-en.tgz


In [ ]:
df.loc[3, "paths"]

'/content/drive/MyDrive/Europarl/el-en.tgz'

In [ ]:
import tarfile
import os

def extract_files(tgz_path):
  path  = os.path.join(tgz_path)
  my_tar = tarfile.open(path)
  my_tar.extractall('./europarl') # specify which folder to extract to
  my_tar.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Extract the files into the europarl dataset
for i in range(19):
  extract_files(df.loc[i, "paths"])
  

Trying out the text extraction with one of the data files in europarl folder

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# split a loaded document into sentences
def to_sentences(doc):
	return doc.strip().split('\n')
 
# shortest and longest sentence lengths
def sentence_lengths(sentences):
	lengths = [len(s.split()) for s in sentences]
	return min(lengths), max(lengths)


In [ ]:
def clean_trigrams(filename):
  

In [ ]:
 
# load English data
filename = '/content/europarl/europarl-v7.fr-en.en'
doc = load_doc(filename)
sentences = to_sentences(doc)
minlen, maxlen = sentence_lengths(sentences)
print('English data: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))
#print(sentences)

 
# # load French data
# filename = 'europarl-v7.fr-en.fr'
# doc = load_doc(filename)
# sentences = to_sentences(doc)
# minlen, maxlen = sentence_lengths(sentences)
# print('French data: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
sentences = []

In [ ]:
# load French data
filename = '/content/europarl/europarl-v7.fr-en.fr'
doc = load_doc(filename)
sentences = to_sentences(doc)
minlen, maxlen = sentence_lengths(sentences)
print('French data: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))

French data: sentences=2007723, min=0, max=693


In [ ]:
# check the sentences
for i in range(10):
  print(sentences[i])

Reprise de la session
Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.
Comme vous avez pu le constater, le grand "bogue de l'an 2000" ne s'est pas produit. En revanche, les citoyens d'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles.
Vous avez souhaité un débat à ce sujet dans les prochains jours, au cours de cette période de session.
En attendant, je souhaiterais, comme un certain nombre de collègues me l'ont demandé, que nous observions une minute de silence pour toutes les victimes, des tempêtes notamment, dans les différents pays de l'Union européenne qui ont été touchés.
Je vous invite à vous lever pour cette minute de silence.
(Le Parlement, debout, observe une minute de silence)
Madame la Présidente, c'est une motion de procédure.
Vous avez probablement appris par la presse et p

# Clean the text



tokenize the set by white space

Normalise the characters to lower case

Remove punctuation from each word

Remove non-printable characters

convert french characters to latin characters

Remove words that contain non-alphabetic characters


In [ ]:
import string
import re
from pickle import dump
from unicodedata import normalize

In [ ]:
def clean_text(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for line in lines:
		# normalize unicode characters
		# line = normalize('NFD', line).encode('ascii', 'ignore')
		# line = line.decode('UTF-8')
		# tokenize on white space
		line = line.split()
		# convert to lower case
		line = [word.lower() for word in line]
		# remove punctuation from each token
		line = [word.translate(table) for word in line]
		# remove non-printable chars form each token
		line = [re_print.sub('', w) for w in line]
		# remove tokens with numbers in them
		line = [word for word in line if word.isalpha()]
		# store as string
		cleaned.append(' '.join(line))
	return cleaned

We will be saving the list of cleaned sentences to a file, that will later be compiled to a dataframe in another notebook. The notebook may not handle such large amount of data 

In [ ]:
# save a list of clean sentences to file
def save_clean_sentences(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

In [ ]:
sentences = clean_text(sentences)
print(len(sentences))

2007723


In [ ]:
# check the sentences
for i in range(10):
  print(sentences[i])

reprise de la session
je declare reprise la session du parlement europeen qui avait ete interrompue le vendredi decembre dernier et je vous renouvelle tous mes vux en esperant que vous avez passe de bonnes vacances
comme vous avez pu le constater le grand bogue de lan ne sest pas produit en revanche les citoyens dun certain nombre de nos pays ont ete victimes de catastrophes naturelles qui ont vraiment ete terribles
vous avez souhaite un debat a ce sujet dans les prochains jours au cours de cette periode de session
en attendant je souhaiterais comme un certain nombre de collegues me lont demande que nous observions une minute de silence pour toutes les victimes des tempetes notamment dans les differents pays de lunion europeenne qui ont ete touches
je vous invite a vous lever pour cette minute de silence
le parlement debout observe une minute de silence
madame la presidente cest une motion de procedure
vous avez probablement appris par la presse et par la television que plusieurs atten

In [ ]:
# save_clean_sentences(sentences, 'french.pkl')

Saved: french.pkl


**Manually deleting all the english files except one**

In [ ]:
len(sentences)

2007723

Hmmm, I am confused..
should i split the data now or should I do so afterwards? after reducing the vocabulary?


In [ ]:
reduced_vocab = []
from collections import Counter

In [ ]:
 
# create a frequency table for all words
def to_vocab(lines):
	vocab = Counter()
	for line in lines:
		tokens = line.split()
		vocab.update(tokens)
	return vocab
 
# remove all words with a frequency below a threshold
def trim_vocab(vocab, min_occurance):
	tokens = [k for k,c in vocab.items() if c >= min_occurance]
	return set(tokens)
 
# mark all OOV with "unk" for all lines
def update_dataset(lines, vocab):
	new_lines = list()
	for line in lines:
		new_tokens = list()
		for token in line.split():
			if token in vocab:
				new_tokens.append(token)
			else:
				new_tokens.append('unk')
		new_line = ' '.join(new_tokens)
		new_lines.append(new_line)
	return new_lines

In [ ]:
reduced_vocab = to_vocab(sentences)
print("vocabulary list:", len(reduced_vocab))
reduced_vocab = trim_vocab(reduced_vocab, 5)
print("Reduced vocab:", len(reduced_vocab))
new_sentences = update_dataset(sentences, reduced_vocab)
print("length of new sentences after reducing vocab:", len(new_sentences))


vocabulary list: 141642
Reduced vocab: 58800
length of new sentences after reducing vocab: 2007723


Okay... That clears it. the length of the sentences will remain same, so train split may not be so problematic


In [ ]:
# check the sentences
for i in range(10):
  print(new_sentences[i])

reprise de la session
je declare reprise la session du parlement europeen qui avait ete interrompue le vendredi decembre dernier et je vous renouvelle tous mes vux en esperant que vous avez passe de bonnes vacances
comme vous avez pu le constater le grand bogue de lan ne sest pas produit en revanche les citoyens dun certain nombre de nos pays ont ete victimes de catastrophes naturelles qui ont vraiment ete terribles
vous avez souhaite un debat a ce sujet dans les prochains jours au cours de cette periode de session
en attendant je souhaiterais comme un certain nombre de collegues me lont demande que nous observions une minute de silence pour toutes les victimes des tempetes notamment dans les differents pays de lunion europeenne qui ont ete touches
je vous invite a vous lever pour cette minute de silence
le parlement debout observe une minute de silence
madame la presidente cest une motion de procedure
vous avez probablement appris par la presse et par la television que plusieurs atten

Wait a min.. does converting to the latin counter part create some kind of translation problems? Darn... Should have omitted the thing

Now that is done with, we need to run the entire thing from the begining. good thing that connection was reset.


Now lets think about the dataframe that will hold the entire trasformation columns and sentences. I think, Dataset can be created from within the previous function itself. Looping through the filenames that was extracted from the tgz files


In [ ]:
import os
for dir_path, dir_names, filenames in os.walk("/content/europarl"):
  for i in filenames:
    # to do map the filenames to the lang identifier list
    


    print("Processing")
    file_path = dir_path + i 
    print(file_path)
    sentences = []
    doc = load_doc(filename)
    sentences = to_sentences(doc)
    print("length of sentences:", len(sentences))
    sentences = clean_text(sentences)
    print("length of sentences after cleaning:", len(sentences))
    # save_clean_sentences(sentences, i+'.pkl')
    # print("saved !", i+'.pkl')
    reduced_vocab = to_vocab(sentences)
    print("vocabulary list:", len(reduced_vocab))
    reduced_vocab = trim_vocab(reduced_vocab, 5)
    print("Reduced vocab:", len(reduced_vocab))
    new_sentences = update_dataset(sentences, reduced_vocab)
    print("length of new sentences after reducing vocab:", len(new_sentences))







Processing
/content/europarleuroparl-v7.lv-en.lv
length of sentences: 2007723
length of sentences after cleaning: 2007723
Saved: europarl-v7.lv-en.lv.pkl
saved ! europarl-v7.lv-en.lv.pkl
Processing
/content/europarleuroparl-v7.sk-en.sk
length of sentences: 2007723
length of sentences after cleaning: 2007723
Saved: europarl-v7.sk-en.sk.pkl
saved ! europarl-v7.sk-en.sk.pkl
Processing
/content/europarleuroparl-v7.de-en.de
length of sentences: 2007723
length of sentences after cleaning: 2007723
Saved: europarl-v7.de-en.de.pkl
saved ! europarl-v7.de-en.de.pkl
Processing
/content/europarleuroparl-v7.et-en.et
length of sentences: 2007723
length of sentences after cleaning: 2007723
Saved: europarl-v7.et-en.et.pkl
saved ! europarl-v7.et-en.et.pkl
Processing
/content/europarleuroparl-v7.el-en.el
length of sentences: 2007723
length of sentences after cleaning: 2007723
Saved: europarl-v7.el-en.el.pkl
saved ! europarl-v7.el-en.el.pkl
Processing
/content/europarleuroparl-v7.lt-en.lt
length of senten